In [1]:
%load_ext autoreload
%autoreload 2
import time
import tensorflow as tf
import p8_util

Using TensorFlow backend.


In [2]:
OUTPUT_DIR = './tmp/baseline'
datadir = './data'
is_tensorboard = False

if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')

In [3]:
# this will kill the processes for Tensorboard
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

In [4]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset)

p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump


In [ ]:
import keras

LEARNING_RATE = 5.e-4
nn_type = 'CNN'
nb_class = nClasses
my_feature_columns, loss_reduction, tf_head = p8_util.get_tf_head("images",tuple_dimension, nClasses)
feature_columns = my_feature_columns

optimizer=tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE)
#optimizer = tf.train.AdagradOptimizer(learning_rate=LEARNING_RATE)
#optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, decay=1e-6, momentum=0.9, nesterov=True)
layer_size = 50
num_layers = 3
learn_mixture_weights = False
dropout = 0.5
seed=p8_util.RANDOM_SEED
is_cnn_batch_norm = True
cnn_layer_config={'feature_map_size':[128]}
initialiser_name = 'xavier'
oNNAdaNetBuilder = p8_util._NNAdaNetBuilder(nn_type, nb_class,feature_columns, optimizer, layer_size, num_layers,
               learn_mixture_weights, dropout, seed, is_cnn_batch_norm,cnn_layer_config, initialiser_name)

In [ ]:
#help(tf.train.get_global_step)

In [ ]:
# Parameters that will be provided to model_fn
BATCH_SIZE = 50  #@param {type:"integer"}
MAX_STEPS = 200
params = {'learning_rate':LEARNING_RATE, 'net_builder':oNNAdaNetBuilder}
NN_TYPE = 'CNN'
my_config = p8_util.make_config(NN_TYPE,output_dir=OUTPUT_DIR, is_restored=False)  
NUM_EPOCHS = 2
classifier = tf.estimator.Estimator(model_fn=p8_util.my_model_fn, params=params, config=my_config)
start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(
    classifier,
    train_spec=tf.estimator.TrainSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train, NUM_EPOCHS\
                          , tuple_dimension=tuple_dimension\
                          , batch_size=BATCH_SIZE),
        max_steps=MAX_STEPS),
    
    eval_spec=tf.estimator.EvalSpec(
        input_fn=p8_util.input_fn("test", x_test, y_test, NUM_EPOCHS\
                           , tuple_dimension=tuple_dimension\
                           , batch_size=BATCH_SIZE),
        steps=None,
        throttle_secs=1))
end_time = time.time()

INFO:tensorflow:Using config: {'_model_dir': './tmp/baseline/CNN', '_tf_random_seed': 42, '_save_summary_steps': 5, '_save_checkpoints_steps': 5, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f41a6896668>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluat

In [ ]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))


In [ ]:
#help(tf.estimator.EstimatorSpec)

In [ ]:
OUTPUT_DIR